In [2]:
from Environment.custom_env import CustomEnvironment
from pettingzoo.test import parallel_api_test
from pettingzoo.mpe import simple_adversary_v3
from Models.ActorCritic import ActorCritic
import json
import torch
import numpy as np
from tqdm import tqdm

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

<function TextIOWrapper.close()>

In [3]:
f = open('Models/model_config.json')
model_variables = json.load(f)
f.close

<function TextIOWrapper.close()>

In [4]:
ac = ActorCritic(model_variables)

In [5]:
env = CustomEnvironment(env_config=env_variables)
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

In [6]:
parallel_api_test(env, num_cycles=1000)

c:\Users\srini\anaconda3\lib\site-packages\pettingzoo\test\parallel_test.py:90: UserWarning: Agent was given info but was dead last turn
  warnings.warn(f"Agent was given {k} but was dead last turn")


agent0 died at time 99
agent2 died at time 99
agent1 died at time 100


c:\Users\srini\anaconda3\lib\site-packages\pettingzoo\test\parallel_test.py:88: UserWarning: Live agent was not given observation
  warnings.warn(f"Live agent was not given {k}")
c:\Users\srini\anaconda3\lib\site-packages\pettingzoo\test\parallel_test.py:88: UserWarning: Live agent was not given info
  warnings.warn(f"Live agent was not given {k}")
c:\Users\srini\anaconda3\lib\site-packages\pettingzoo\test\parallel_test.py:90: UserWarning: Agent was given terminated but was dead last turn
  warnings.warn(f"Agent was given {k} but was dead last turn")


agent2 consumed pellet57
agent2 consumed pellet10
agent0 died at time 99
agent1 died at time 100
agent2 died at time 142
Passed Parallel API test


In [7]:
observations, infos = env.reset()
while env.agents:
    # print(env.timestep)
    # this is where you would insert your policy
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}
    
    observations, rewards, terminations, truncations, infos = env.step(actions)

# env.close()

agent0 died at time 99
agent2 died at time 99
agent1 died at time 100


In [8]:
state = torch.ones((3, 10, 10))

episodes = 100

for i in tqdm(range(episodes)): #episodes loop"
    curr_obs,infos = env.reset() #reset environment after each episode\n",
    for j in range(100):
        actions = {}
        for key,value in objects.items(): #objects contains all of the models, get the corresponding actions from each policy\n",
            act,log_prob = value.select_action(state = curr_obs[key]) #action of for each agent is set based on the current state\n",
            actions[key] = act

        observations,rewards,terminations,truncations,infos = env.step(actions)

        ac_agent.push_to_buffer(curr_obs[0],actions[0],rewards[0],observations[0])

  0%|          | 0/100 [00:00<?, ?it/s]


NameError: name 'objects' is not defined